In [1]:
import torch 
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

x=torch.randn(2,20)
net(x)

tensor([[-0.1171, -0.0949,  0.0158, -0.1308, -0.0257, -0.0938,  0.1530,  0.1321,
         -0.1618,  0.2842],
        [-0.0545, -0.3102,  0.1217,  0.1255, -0.0719, -0.1989, -0.0387,  0.0787,
         -0.2609,  0.2978]], grad_fn=<AddmmBackward0>)

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)  #隐藏层
        self.out=nn.Linear(256,10)  #输出层

    #前向传播
    def forward(self,x):
        return self.out(F.relu(self.hidden(x)))
    
net=MLP()
net(x)

tensor([[ 0.2375, -0.0538,  0.3295, -0.0302,  0.0306, -0.2587, -0.0179,  0.2673,
          0.0804, -0.0471],
        [-0.1105, -0.0811,  0.0849,  0.2786, -0.5719,  0.2603,  0.0912, -0.1008,
          0.0082,  0.2994]], grad_fn=<AddmmBackward0>)

顺序块

In [3]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            #变量 modules 中。module 的类型是OrderedDict，
            self._modules[str(idx)] = module

    def forward(self, x):
        #OrderedDict保证了按照成员添加的顺序遍历他们
        for block in self._modules.values():
            x = block(x)
        return x
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(x)

tensor([[-0.1361,  0.3502,  0.0180, -0.3931,  0.2791,  0.0783,  0.2435, -0.5411,
          0.5887, -0.5729],
        [-0.4164,  0.0588,  0.1979, -0.0213, -0.0866, -0.3862, -0.2877, -0.1859,
          0.0369, -0.4005]], grad_fn=<AddmmBackward0>)

In [4]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.randn((20, 20),requires_grad=False)
        self.liner=nn.Linear(20, 20)

    def forward(self,x):
        x=self.liner(x)
        #使用创建的常量参数以及 relu 和 mm 函数
        x=F.relu(torch.mm(x, self.rand_weight)+1)
        #复用全连接层。这相当于两个全连接层共享参数
        x=self.liner(x)
        #控制流
        while x.abs().sum()>1:
            x/=2
        return  x.sum()
    
net=FixedHiddenMLP()
net(x)



tensor(-0.0738, grad_fn=<SumBackward0>)

In [5]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(20,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU()
        )
        self.linear=nn.Linear(32,16)

    def forward(self,x):
        return self.linear(self.net(x))
    
chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(x)


tensor(0.0965, grad_fn=<SumBackward0>)